In [92]:
import numpy as np
import pandas as pd
import requests
import json
import dotenv
import os

## Getting Biographical Data from the Congress API

The most important skill is reading the API documentation efficiently.

Website: https://gpo.congress.gov

Look ONLY for these five things:

1. Root: the beginning of all URLs associated with this API for getting data back (I often find this in examples first)

https://api.congress.gov/v3

2. Endpoints: the second part of the URL, generally it points to a specific data set to be returned

/member/{bioguideId}

The curly braces mean fill this in for whichever member of congress you want. 

3. The headers: this is data that WE give to the API to identify ourselves, and provide other information if requested
* User Agent string: a string that identifies your software. If you are writing a Bot, what is your bot's name, and what methods are using to access this API.

The standard convention is: 

'botname/version (email) python-requests/request-package-version'

In [93]:
botname = 'targ'
version = '0.0'
email = 'hxg9uy@virginia.edu'
useragent = f'{botname}/{version} ({email}) python-requests/{requests.__version__}'
headers = {'User-Agent': useragent}

4. API Parameters: these are ways to control what data is requested. The parameters are generally listed in the documentation for each endpoint. 3 kinds of parameters:

* Query parameters: these should be listed in a dictionary and passed to the `params` argument of `requests.get()` 

* Path parameters: are part of the endpoint (such as bioguideID)

* Global parameters: apply to any use of the API regardless of endpoint. Often that includes your API key. Can be in either the headers or the params

5. If an API key is needed, how to get one, and how will you let the API know the key?

API keys should be saved in a .env file in the same folder as your code. Use `dotenv` package to load the key into yoru code without ever displaying/exposing your key. 

In [94]:
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')

In [95]:
params = {'format': 'json', 
          'api_key': congresskey}

Put it all together

In [96]:
bioguideId = 'M001239'
root = 'https://api.congress.gov/v3'
endpoint = f'/member/{bioguideId}'

r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)
r

<Response [200]>

In [97]:
r.json()

{'member': {'addressInformation': {'city': 'Washington',
   'district': 'DC',
   'officeAddress': '1013 Longworth House Office Building',
   'phoneNumber': '(202) 225-4711',
   'zipCode': 20515},
  'bioguideId': 'M001239',
  'birthYear': '1968',
  'cosponsoredLegislation': {'count': 168,
   'url': 'https://api.congress.gov/v3/member/M001239/cosponsored-legislation'},
  'currentMember': True,
  'depiction': {'attribution': 'Image courtesy of the Member',
   'imageUrl': 'https://www.congress.gov/img/member/67744ba20b34857ecc909149_200.jpg'},
  'directOrderName': 'John J. McGuire III',
  'district': 5,
  'firstName': 'John',
  'honorificName': 'Mr.',
  'invertedOrderName': 'McGuire, John J.',
  'lastName': 'McGuire',
  'middleName': 'J.',
  'officialWebsiteUrl': 'https://mcguire.house.gov/',
  'partyHistory': [{'partyAbbreviation': 'R',
    'partyName': 'Republican',
    'startYear': 2025}],
  'previousNames': [{'directOrderName': 'John McGuire',
    'endDate': '2025-01-10T17:36:18Z',
   

## Sponsored Legislation

In [98]:
endpoint = f'/member/{bioguideId}/sponsored-legislation'
endpoint



'/member/M001239/sponsored-legislation'

In [99]:
params = {'format': 'json',
          'offset': 0,
          'limit': 250,
          'api_key': congresskey}

In [100]:
r = requests.get(root + endpoint,
                    headers=headers,
                    params=params)
r

<Response [200]>

In [101]:
myjson = r.json()
myjson

{'pagination': {'count': 9},
 'request': {'bioguideId': 'm001239',
  'contentType': 'application/json',
  'format': 'json'},
 'sponsoredLegislation': [{'congress': 119,
   'introducedDate': '2025-09-26',
   'latestAction': {'actionDate': '2025-09-26',
    'text': 'Referred to the House Committee on Ways and Means.'},
   'number': '5595',
   'policyArea': {'name': None},
   'title': 'To amend the Internal Revenue Code of 1986 to modify the tax on remittance transfers.',
   'type': 'HR',
   'url': 'https://api.congress.gov/v3/bill/119/hr/5595?format=json'},
  {'congress': 119,
   'introducedDate': '2025-09-19',
   'latestAction': {'actionDate': '2025-09-19',
    'text': 'Referred to the House Committee on Armed Services.'},
   'number': '5522',
   'policyArea': {'name': None},
   'title': 'To change the name of the Department of Defense to the "Department of War", and for other purposes.',
   'type': 'HR',
   'url': 'https://api.congress.gov/v3/bill/119/hr/5522?format=json'},
  {'congres

In [102]:
myjson['pagination']['count']

9

In [103]:
myjson['sponsoredLegislation']

sponsoredLegislation_df = pd.json_normalize(myjson, record_path = 'sponsoredLegislation') 

In [104]:
for x in sponsoredLegislation_df['title']:
    print(x)

To amend the Internal Revenue Code of 1986 to modify the tax on remittance transfers.
To change the name of the Department of Defense to the "Department of War", and for other purposes.
Border Wall Status Act
Make the District of Columbia Safe and Beautiful Act
FAIR Act
VA Data Transparency and Trust Act
Manned Aircraft Clarification Act
Uranium for Energy Independence Act of 2025
Agricultural and Forestry Hauling Efficiency Act


In [105]:
r = requests.get(sponsoredLegislation_df['url'][3],
                headers=headers,
                params=params)

r

<Response [200]>

In [106]:
r.json()

{'bill': {'actions': {'count': 6,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/actions?format=json'},
  'committees': {'count': 2,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/committees?format=json'},
  'congress': 119,
  'constitutionalAuthorityStatementText': '<pre>\n[Congressional Record Volume 171, Number 144 (Wednesday, September 3, 2025)]\n[House]\nFrom the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]\nBy Mr. McGUIRE:\nH.R. 5103.\nCongress has the power to enact this legislation pursuant\nto the following:\nArticle I, Section 8\n[Page H3829]\n</pre>',
  'cosponsors': {'count': 3,
   'countIncludingWithdrawnCosponsors': 3,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/cosponsors?format=json'},
  'introducedDate': '2025-09-03',
  'latestAction': {'actionDate': '2025-09-10',
   'text': 'Ordered to be Reported (Amended) by the Yeas and Nays: 25 - 19.'},
  'legislationUrl': 'https:

## FEC Data for Financial Campaign Contributions

In [107]:
#dotenv.lead_dotenv()
feckey = os.getenv('feckey')

In [113]:
root = 'https://api.open.fec.gov/'
endpoint = '/v1/candidates/search'
params = {'api_key': feckey,
          'q': 'John McGuire',
          'state': 'VA',
          'district': '5',
          'office': 'H'}
r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)

r



<Response [200]>

In [119]:
myjson = r.json()
candidate_id = myjson['results'][0]['candidate_id']
candidate_id

'H0VA07133'

In [125]:
endpoint = f'/v1/schedules/schedule_a/'
params = {'api_key': feckey,
          'candidate_id': candidate_id,
          'two_year_transaction_period': '2026',
          }

r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)

r

<Response [200]>

In [126]:
r.json()

{'api_version': '1.0',
 'pagination': {'count': 52584569,
  'is_count_exact': False,
  'last_indexes': {'last_index': '4091020251290543084',
   'sort_null_only': True},
  'pages': 2629229,
  'per_page': 20},
 'results': [{'amendment_indicator': 'A',
   'amendment_indicator_desc': 'ADD',
   'back_reference_schedule_name': None,
   'back_reference_transaction_id': None,
   'candidate_first_name': None,
   'candidate_id': None,
   'candidate_last_name': None,
   'candidate_middle_name': None,
   'candidate_name': None,
   'candidate_office': None,
   'candidate_office_district': None,
   'candidate_office_full': None,
   'candidate_office_state': None,
   'candidate_office_state_full': None,
   'candidate_prefix': None,
   'candidate_suffix': None,
   'committee': {'affiliated_committee_name': 'APPLIED MATERIALS, INC.',
    'candidate_ids': [],
    'city': 'BURLINGAME',
    'committee_id': 'C00406892',
    'committee_type': 'Q',
    'committee_type_full': 'PAC - Qualified',
    'cycle': 2

In [ ]:
root = 'https://api.open.fec.gov/'
endpoint = '/v1/candidate/'

